# Word Embeddings: Encoding Lexical Semantics

Author: YinTaiChen

In [1]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
word_to_ix = {"hello": 0, "world": 1}
embeds = nn.Embedding(2, 5)
lookup_tensor =  torch.LongTensor([word_to_ix["hello"]])
hello_embed = embeds(autograd.Variable(lookup_tensor))
print(hello_embed)

Variable containing:
-0.9851 -1.3483 -0.7949  2.3588  1.7596
[torch.FloatTensor of size 1x5]



# An Example: N-Gram Language Modeling

In [3]:
CONTEXT_SIZE = 2 # use two context words to predict the target
EMBEDDING_DIM = 10 # the embedded word has dimension of 10

test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()

trigrams = [([test_sentence[i], test_sentence[i+1]], test_sentence[i+2]) for i in range(len(test_sentence) - 2)]
# trigrams is a list of (context, target) tuple
print(trigrams[:3])

vocab = set(test_sentence)
# vocab is a set of the words in test sentence
word_to_ix = {word: i for i, word in enumerate(vocab)}
# a dict of string: integer

[(['When', 'forty'], 'winters'), (['forty', 'winters'], 'shall'), (['winters', 'shall'], 'besiege')]


In [4]:
class NGramLanguageModeler(nn.Module):
    
    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramLanguageModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)
        
    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs

In [5]:
losses = []
loss_function = nn.NLLLoss()
model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.001)

for epoch in range(10):
    total_loss = torch.Tensor([0])
    for context, target in trigrams:
        context_idxs = [word_to_ix[w] for w in context]
        context_var = autograd.Variable(torch.LongTensor(context_idxs))
        model.zero_grad()
        log_probs = model(context_var)
        loss = loss_function(log_probs, autograd.Variable(torch.LongTensor([word_to_ix[target]])))
        loss.backward()
        optimizer.step()
        total_loss += loss.data
    losses.append(total_loss)
print(losses)

[
 520.6655
[torch.FloatTensor of size 1]
, 
 518.3842
[torch.FloatTensor of size 1]
, 
 516.1178
[torch.FloatTensor of size 1]
, 
 513.8633
[torch.FloatTensor of size 1]
, 
 511.6204
[torch.FloatTensor of size 1]
, 
 509.3881
[torch.FloatTensor of size 1]
, 
 507.1661
[torch.FloatTensor of size 1]
, 
 504.9525
[torch.FloatTensor of size 1]
, 
 502.7479
[torch.FloatTensor of size 1]
, 
 500.5507
[torch.FloatTensor of size 1]
]
